## Assignment-07 First Step of using machine learning and models.

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [1]:
import pandas as pd
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [2]:
fname = '/home/dk/kkb/ai-for-nlp-course-assignment/sqlResult_1558435.csv'
df = pd.read_csv(fname, encoding='gb18030')

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [4]:
#source 和　content 为 NaN 的需要删除
#df[df.isna()['content']]
#df[df.isna()['source']]

df = df.dropna(subset=['source', 'content'])
#新华社新闻所占的比例
print (len(df[df['source'] == '新华社']) / len(df))

0.903609336948031


In [5]:
#source列根据是“新华社”转化为label
df['source'] = df['source'].apply(lambda x : 1 if x == '新华社' else 0)

In [6]:
#数据清洗
#先尝试一下分词,去停用词的方案看下效果
def load_stopwords(fpath):
    return set([w.strip() for w in open(fpath, encoding='utf-8').readlines()])
stopwords = load_stopwords('./stopwords.txt') 

In [15]:
pattern = r"[a-zA-Z\d\u3000\\n\r\n]" #数字字母等全部去除　\u3000中文空格符
pattern = re.compile(pattern)
corpus = []
p_ids = []
for i, row in enumerate(zip(df['id'], df['content'])):
    #if pd.isna(p): continue   #判断是否是空值
    p_id, p = row
    p = pattern.sub('', p)
    p = ' '. join([w for w in jieba.cut(p) if w not in stopwords])
    #p = re.sub('(\d )+', '#', p)
    corpus.append(p)
    p_ids.append(p_id)
    if i % 10000 == 0:
        print ('processing {} paragraphs.'.format(i))

processing 0 paragraphs.
processing 10000 paragraphs.
processing 20000 paragraphs.
processing 30000 paragraphs.
processing 40000 paragraphs.
processing 50000 paragraphs.
processing 60000 paragraphs.
processing 70000 paragraphs.
processing 80000 paragraphs.


### Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

对文本进行向量化

In [24]:
corpus = corpus
vectorizer = TfidfVectorizer(max_features=50000)
X = vectorizer.fit_transform(corpus)

In [25]:
y = df['source'].values

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

In [79]:
X_train, X_test, y_train, y_test, p_ids_train, p_ids_test = train_test_split(X, y, p_ids, test_size=0.4, random_state=42)

In [80]:
X_val, X_test, y_val, y_test, p_ids_val, p_ids_test = train_test_split(X_test, y_test, p_ids_test, test_size=0.5, random_state=42)

In [81]:
print ("train data shape {}".format(X_train.shape))
print ("validation data shape {}".format(X_val.shape))
print ("test data shape {}".format(X_test.shape))

train data shape (52231, 50000)
validation data shape (17410, 50000)
test data shape (17411, 50000)


In [82]:
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [31]:
neigh.score(X_val, y_val)

0.9350947731188972

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

In [83]:
train_predict = neigh.predict(X_train)
val_predict = neigh.predict(X_val)
test_predict = neigh.predict(X_test)

In [84]:
target_names=["非新华社","新华社"]
print ("train data")
print(classification_report(y_train, train_predict, target_names=target_names))
print("***********************************************************************")
print ("val data")
print(classification_report(y_val, val_predict, target_names=target_names))
print("***********************************************************************")
print ("test data")
print(classification_report(y_test, test_predict, target_names=target_names))

train data
              precision    recall  f1-score   support

        非新华社       1.00      1.00      1.00      5075
         新华社       1.00      1.00      1.00     47156

    accuracy                           1.00     52231
   macro avg       1.00      1.00      1.00     52231
weighted avg       1.00      1.00      1.00     52231

***********************************************************************
val data
              precision    recall  f1-score   support

        非新华社       0.85      0.37      0.52      1630
         新华社       0.94      0.99      0.97     15780

    accuracy                           0.94     17410
   macro avg       0.89      0.68      0.74     17410
weighted avg       0.93      0.94      0.92     17410

***********************************************************************
test data
              precision    recall  f1-score   support

        非新华社       0.90      0.37      0.52      1686
         新华社       0.94      1.00      0.97     15725

    accu

### Step6: 使用 test_data 对其进行新数据的判断, 你有没

In [85]:
# it's your time

### Step7: 调整不同的参数，观察变化

对“非新华社”也即label为０的召回率比较低，那么可以增加“非新华社”的权重,比如重复“非新华社”的样本；

In [86]:
# it's your time

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

对这个问题，是validation上的“新华社”的召回率要越高越好，当然同时其precision也要比较高，这样就不会把“抄袭”的文章给漏掉！

### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

需要把实际数据的id给带上，如下判断抄袭的时候，才能知道具体的数据是什么，然后做分析：

In [87]:
print (y_test.shape)
print (test_predict.shape)
print (len(p_ids_test))

(17411,)
(17411,)
17411


In [88]:
p_ids_final=[] #最终预测为１，实际为０的文章id list
for i, v in enumerate(zip(y_test, test_predict)):
    if v[0] == 0 and v[1] ==1:
        p_ids_final.append(p_ids_test[i])
    

In [90]:
len(p_ids_final)

1068

In [93]:
df_result = df[df['id'].isin(p_ids_final)]

In [94]:
df_result

,id,author,source,content,feature,title,url
6,89611,NaN,0,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""21""...",配骁龙660 全面屏鼻祖夏普新机酝酿中,http://www.cnbeta.com/articles/tech/623603.htm
32,89585,NaN,0,其他规格方面，搭载飞思卡尔IMX6SL处理器，内存512MB，存储8GB，采用定制版系统，低...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""64""...",999元 - QQ阅读发布首款电子书：支持QQ微信登录,http://www.cnbeta.com/articles/tech/623613.htm
52,89565,NaN,0,以下是文章主要内容：\r\n不妨想象一下未来的这一场景：你走进商店，凭名字通过带有脸部识别功...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""6"",...",亚马逊收购全食超市或预示着排队结账时代结束,http://www.cnbeta.com/articles/tech/623629.htm
53,89564,NaN,0,央广网贵阳6月19日消息（记者王珩 贵州台记者黄瑾）为规范省级救灾储备物资管理，提高救灾物资...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",贵州省出台省级救灾储备物资管理办法规定 灾情发生24小时内省级救灾物资须转运至目的地,http://china.huanqiu.com/hot/2017-06/10866404....
54,89563,李天奕_NN7528,0,（原标题：高速上50秒别车6次 面包车司机现身：我一时冲动犯了错）\r\n高速上50秒别车6...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1826""...",司机报复后车高速上50秒别车6次 被扣2分罚款2,http://news.163.com/17/0606/17/CM91M78000018AO...
75,89542,NaN,0,虽然飞行时间只有八分钟左右，但从大约4分钟开始，火箭将部署10个胶囊。每个胶囊的大小都是可乐...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""4"",...",美国宇航局安排多彩火箭发射庆祝父亲节,http://www.cnbeta.com/articles/tech/623645.htm
80,89537,NaN,0,新疆日报讯（记者覃超祥报道）6月14日，玉山·买买提像往常一样来到新疆农业大学家属院7号楼，...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",【民族团结一家亲】玉山·买买提：用爱心善举回报社会,http://china.huanqiu.com/hot/2017-06/10866435....
83,89534,NaN,0,哈登和保罗联手，火箭会变成什么样子？这或许是一个非常疯狂的想法，毕竟快船正在用2亿顶薪去诱惑...,"{""type"":""体育"",""site"":""凤凰"",""commentNum"":""30"",""jo...",火箭可砸1.5亿打劫保罗？哈登CP3能打破勇士骑士垄断,http://sports.ifeng.com/a/20170616/51262355_0....
87,89530,NaN,0,虽说没啥内容，可这则TVC将会在ICC期间播放，要是你知道印度人民对板球的喜好，自然会知道有...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",[动图]一加手机5真容曝光 真的不像是iPhone,http://www.cnbeta.com/articles/tech/623653.htm
88,89529,NaN,0,中国网财经6月23日讯 午后粤港澳概念股走俏，截至发稿，珠海港涨近4%，恒基达鑫涨近3%，中...,"{""type"":""中国财经"",""site"":""参考消息"",""commentNum"":""0"",...",快讯：粤港澳概念股走俏 珠海港涨近4%,http://www.cankaoxiaoxi.com/finance/20170623/2...


### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。